In [12]:
import numpy as np
from pathlib import Path
import sys

In [13]:
class TreeNode:
    def __init__(self, level, V, W, taken):
        self.level = level # niveau dans l’arbre de recherche >=0
        self.V = V # valeur de la solution courante
        self.W = W # poids de la solution courante
        self.taken = taken # liste des index d’objets de la solution courante
    def __str__(self):
        return str((self.level, self.V, self.W, self.taken))

In [14]:
class KPBB:
    def __init__(self, cap, values, weights): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.unordered = [((v, w), i) for i, (v, w) in enumerate(zip(self.values, self.weights))]
        self.ordered = sorted([((v, w), i) for i, (v, w) in enumerate(zip(self.values, self.weights))], key = lambda tup: float(tup[0][0])/tup[0][1], reverse = True)
        self.low_band=0

    def initialSolution(self): # calcul de la solution initiale
        for i in self.ordered:
            if i[0][1]<=self.capacity:
                self.low_band+=i[0][0]
                self.capacity-=i[0][1]
        return self.low_band
    
    def funcUB(self,node):
        UB=node.values
        Ww=np.copy(node.W) 
        i=np.copy(node.level)
        for n in np.array(self.ordered[node.level,:]):
            if self.weights[i]<=n.W:
                 Ww-=self.weights[i]
                 UB+=self.values[i]
            else:
                Ww-=self.weights[i] 
                UB+=self.values[i]            

    def nodeEvaluation(self,index, node): # fonction d’évaluation d’un noeud
        pass

    def solve(self): # algorithme de résolution
        pass

In [ ]:
class TreeNode:
    """Classe pour les noeuds de l'arbre de recherche. Chaque noeud est caractérisé par : 
    - son niveau dans l'arbre de recherche (>= 0)
    - la valeur de la solution courante
    - le poids de la solution courante
    - la liste des index d'objets de la solution courante
    """

    def __init__(self, level, V, W, taken):
        self.level = level  # niveau dans l'arbre de recherche >= 0
        self.V = V          # valeur de la solution courante
        self.W = W          # poids de la solution courante
        self.taken = taken  # liste des index d'objets de la solution courante

    def __str__(self):
        return str((self.level, self.V, self.W, self.taken)) # affichage d'un noeud
    

class KPBB:
    """Classe pour le problème du sac à dos avec l'algorithme de branch and bound.
    - cap : capacité du sac à dos
    - values : liste des valeurs des objets
    - weights : liste des poids des objets 
    - unordered : liste des objets avec leur index
    - ordered : liste des objets triés par rapport à leur ratio valeur/poids
    """
    def __init__(self, cap, values, weights): # Initialisation des données
        self.cap = cap
        self.values = values
        self.weights = weights
        self.unordered = [((v,w), i) for i, (v,w) in enumerate(zip(self.values, self.weights))] # liste des objets avec leur index
        self.ordered = sorted([((v,w),i) for i, (v,w) in enumerate(zip(self.values, self.weights))], key=lambda tup: tup[0][0]/tup[0][1], reverse=True) # liste des objets triés par rapport à leur ratio valeur/poids.


    # calcul de la solution initiale
    def initial_solution(self):
        taken = []
        V = 0
        W = 0
        for (v, w), i in self.ordered:
            if W + w <= self.cap:
                taken.append(i)
                V += v
                W += w
        return V, W, taken


    # fonction d'évaluation d'un noeud
    def nodeEvaluation(self, index, node):
        V, W = node.V, node.W
        for i in range(index, len(self.ordered)):
            (v, w), j = self.ordered[i]
            if W + w <= self.cap:
                V += v
                W += w
            else:
                V += (self.cap - W) * v / w
                break
        return V


    def solve(self):
        best = TreeNode(-1, 0, 0, []) 
        racine = TreeNode(0, 0, 0, []) 
        queue = [racine] 
        
        while queue: 
            node = queue.pop() 
            #si la valeur de la solution courante est meilleure que la meilleure solution je mets à jour la meilleure solution
            if node.level == len(self.ordered): 
                if node.V > best.V: 
                    best = node  
            else:
                i = node.level
                (v, w), j = self.ordered[i]
                if node.W + w <= self.cap:
                    taken = node.taken + [j]
                    V = node.V + v
                    W = node.W + w
                    enfant = TreeNode(i+1, V, W, taken)
                    if enfant.V > best.V:
                        queue.append(enfant)
                if self.nodeEvaluation(i+1, node) > best.V:
                    queue.append(TreeNode(i+1, node.V, node.W, node.taken))
        return best.V, best.W, best.taken


In [18]:

cap = 10 
values = [40, 50, 100, 95, 30] 
poids = [2, 3, 1, 5, 3]
kpbb = KPBB(cap, values, poids)
V, W, taken = kpbb.solve()
print("Valeur optimale :", V)
print("Poids optimal :", W) 
print("Objets pris :", taken)
print("Valeurs des objets pris :", [values[i] for i in taken])


Valeur optimale : 245
Poids optimal : 9
Objets pris : [2, 3, 1]
Valeurs des objets pris : [100, 95, 50]
